En este notebook se recogen varias pruebas solo con los contadores cuyas series temporales son completas o casi completas.

**PRUEBAS HECHAS**
1. Predicción haciendo la media entre XGBoost Regressor y Gradient Boosting Regressor (BASELINE ENTREGA LOCAL)
2. El 1 pero hacemos un modelo agregado por semanas para las predicciones agregadas semanales (EMPEORA)
3. Meter variable lag_7 y lag_14 (MEJORA)
4. Hacer selección de variables atendiendo a las features_importances_ de XGBoost (MEJORA sospechosamente MUCHO)

**PRUEBAS POR HACER**
- Usar de train menos meses
- Quitar predicciones negativas (ponerlas a 0 y a funcionar)
- Usar catboost en vez de xgboost
- Probar a hacer un modelo agregado semanal pero usando ARIMA
- Probar modelo diario con ARIMA (solo en contadores buenos)
- Quitar outlayers (esto es más en preprocesado)
- Hacer logaritmos a (casi) todo
- LSTM (a saber qué se puede gestionar con esto) (puede ser buena idea no usar toda la serie temporal)

### Imports, utils and train/test creation

In [86]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
import warnings

In [60]:
warnings.simplefilter('ignore')

In [61]:
'''
given a start date in datetime format "start_date" and an "end_date" returns a list of strings with the dates from
"start_date" to "end_date".

Example:

start_date = datetime.date(2019, 9 , 30)
end_date = datetime.date(2019, 10, 7)
get_date_range(start_date, end_date)
'''
def get_date_range(start_date, end_date):
    number_of_days = (end_date-start_date).days
    return [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days+1)]

'''
This function expects two dataframes with the same format: for the first seven columns, each column corresponds to a date 
and each row corresponds to a counter index. In position i,j there should be DELTA of counter i in date j. 
For the last two columns of the dataframes they should not reffer to a daily prediction but to the aggregated prediction 
of week_1 and week_2. Given these two dataframes (one for theprediction and one for the real values), 
the function returns de error according to the competition rules.

Examples:

import pandas as pd
import copy

test = pd.read_pickle('../data/test.pkl')

compute_error(test, test)

test_v3 = copy.copy(test)
test_v3.iloc[:,0] = test_v3.iloc[:,1]
compute_error(test_v3, test)

'''
def compute_error(pred, real):
    daily_rmses = []
    for i in range(7):
        daily_rmses.append((((real.iloc[:,i] - pred.iloc[:,i])**2/len(real.iloc[:,i])).sum())**(1/2))
    rmse_1 = sum(daily_rmses)/7
    
    first_week_pred_sum = pred.iloc[:,7].sum()
    second_week_pred_sum = pred.iloc[:,8].sum()
    first_week_real_sum = real.iloc[:,7].sum()
    second_week_real_sum = real.iloc[:,8].sum()
    
    first_week_rmse = (((first_week_real_sum - first_week_pred_sum)**2)/len(real.iloc[:,7]))**(1/2)
    second_week_rmse = (((second_week_real_sum - second_week_pred_sum)**2)/len(real.iloc[:,8]))**(1/2)
    rmse_2 = (first_week_rmse + second_week_rmse)/2
    
    return (rmse_1 + rmse_2)/2

In [62]:
path = '../data/df6.pkl'

df = pd.read_pickle(path)
start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 17)
train = df[df['DATE'].isin(get_date_range(start_date, end_date))]
train = train[train['IS_GOOD']==1]
train.drop(['YEAR_DAY','WEEKDAY','IS_GOOD','DATE'], axis=1, inplace=True)
train['SUN'] = train['SUN'].fillna(train['SUN'].mean())
train['PRECIPITATIONS'] = train['PRECIPITATIONS'].fillna(train['PRECIPITATIONS'].mean())

start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
test = df[df['DATE'].isin(get_date_range(start_date, end_date))]
test = test[test['IS_GOOD']==1]
test.drop(['YEAR_DAY','WEEKDAY','IS_GOOD'], axis=1, inplace=True)
test['SUN'] = test['SUN'].fillna(test['SUN'].mean())
test['PRECIPITATIONS'] = test['PRECIPITATIONS'].fillna(test['PRECIPITATIONS'].mean())

print('Train:', train.shape, 'Test:', test.shape)

X_train = train.drop(['DELTA'], axis=1)
y_train = train['DELTA']

X_test = test.drop(['DELTA', 'DATE'], axis=1)

Train: (931203, 14) Test: (37142, 15)


### XGBR and GBR for all counters

- The final prediction is the mean between XGBR and GBR
- No lags are used
- The week prediction is done just by adding the daily predictions 

In [63]:
path = '../data/df6.pkl'

df = pd.read_pickle(path)
start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 17)
train = df[df['DATE'].isin(get_date_range(start_date, end_date))]
train.drop(['YEAR_DAY','WEEKDAY','IS_GOOD','DATE'], axis=1, inplace=True)
train['SUN'] = train['SUN'].fillna(train['SUN'].mean())
train['PRECIPITATIONS'] = train['PRECIPITATIONS'].fillna(train['PRECIPITATIONS'].mean())

start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
test = df[df['DATE'].isin(get_date_range(start_date, end_date))]
test.drop(['YEAR_DAY','WEEKDAY','IS_GOOD'], axis=1, inplace=True)
test['SUN'] = test['SUN'].fillna(test['SUN'].mean())
test['PRECIPITATIONS'] = test['PRECIPITATIONS'].fillna(test['PRECIPITATIONS'].mean())

print('Train:', train.shape, 'Test:', test.shape)

X_train = train.drop(['DELTA'], axis=1)
y_train = train['DELTA']

X_test = test.drop(['DELTA', 'DATE'], axis=1)

Train: (964197, 14) Test: (38458, 15)


In [64]:
model1 = xgb.XGBRegressor(
    n_estimators=1000,
    reg_lambda=1,
    gamma=0,
    max_depth=8
)

model2 = GradientBoostingRegressor()

print('Fitting XGB...')
model1.fit(X_train, y_train)
print('Fitting GB...')
model2.fit(X_train, y_train)
print('End fitting.')

Fitting XGB...
Fitting GB...
End fitting.


In [65]:
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)

results_df = pd.DataFrame.from_dict({'ID':test['ID'].values, 
                                     'DATE':test['DATE'].values,
                                     'y_pred1':y_pred1,
                                     'y_pred2':y_pred2})
results_df = results_df.sort_values(['ID','DATE'])
results_df['FINAL'] = results_df[['y_pred1','y_pred2']].mean(axis=1)

In [66]:
start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
fechas_test = get_date_range(start_date, end_date)

ID = []
Dia_1 = []
Dia_2 = []
Dia_3 = []
Dia_4 = []
Dia_5 = []
Dia_6 = []
Dia_7 = []
for i, fecha in enumerate(fechas_test[0:7]):
    aux = results_df[results_df['DATE']==fecha]
    ID = list(aux['ID'].values)
    if i==0:
        Dia_1 += list(aux['FINAL'].values)
    if i==1:
        Dia_2 += list(aux['FINAL'].values)
    if i==2:
        Dia_3 += list(aux['FINAL'].values)
    if i==3:
        Dia_4 += list(aux['FINAL'].values)
    if i==4:
        Dia_5 += list(aux['FINAL'].values)
    if i==5:
        Dia_6 += list(aux['FINAL'].values)
    if i==6:
        Dia_7 += list(aux['FINAL'].values)
print(len(ID),len(Dia_1))
final_df = pd.DataFrame.from_dict({'ID':ID,
                                   'Dia_1':Dia_1,
                                  'Dia_2':Dia_2,
                                  'Dia_3':Dia_3,
                                  'Dia_4':Dia_4,
                                  'Dia_5':Dia_5,
                                  'Dia_6':Dia_6,
                                  'Dia_7':Dia_7,})

ID = []
Dia_8 = []
Dia_9 = []
Dia_10 = []
Dia_11 = []
Dia_12 = []
Dia_13 = []
Dia_14 = []
for i, fecha in enumerate(fechas_test[7:14]):
    aux = results_df[results_df['DATE']==fecha]
    ID = list(aux['ID'].values)
    if i==0:
        Dia_8 += list(aux['FINAL'].values)
    if i==1:
        Dia_9 += list(aux['FINAL'].values)
    if i==2:
        Dia_10 += list(aux['FINAL'].values)
    if i==3:
        Dia_11 += list(aux['FINAL'].values)
    if i==4:
        Dia_12 += list(aux['FINAL'].values)
    if i==5:
        Dia_13 += list(aux['FINAL'].values)
    if i==6:
        Dia_14 += list(aux['FINAL'].values)
print(len(ID),len(Dia_11))
final_df2 = pd.DataFrame.from_dict({'ID':ID,
                                   'Dia_8':Dia_8,
                                  'Dia_9':Dia_9,
                                  'Dia_10':Dia_10,
                                  'Dia_11':Dia_11,
                                  'Dia_12':Dia_12,
                                  'Dia_13':Dia_13,
                                  'Dia_14':Dia_14,})

final_df['Semana_1'] = final_df[['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']].sum(axis=1)
final_df['Semana_2'] = final_df2[['Dia_8','Dia_9','Dia_10','Dia_11','Dia_12','Dia_13','Dia_14']].sum(axis=1)

final_df2 = final_df.drop('ID', axis=1)

2747 2747
2747 2747


In [67]:
test = pd.read_pickle('../data/test.pkl')
error = compute_error(final_df2, test)
print('Mean between XGBR and GBR:', round(error,2))

Mean between XGBR and GBR: 21594.29


### Train and test sets for weekly predictions instead of adding daily predictions

In [130]:
path = '../data/df6.pkl'

df = pd.read_pickle(path)
start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 17)
week_train = df[df['DATE'].isin(get_date_range(start_date, end_date))]
week_train['YEAR_WEEK'] = (week_train['YEAR_DAY']-1)//7
week_train = week_train[week_train['YEAR_WEEK']!=-1]
week_train.drop(['YEAR_DAY','IS_WEEKEND','WEEKDAY','sin_WEEKDAY','cos_WEEKDAY',
                 'sin_year_day','cos_year_day','IS_GOOD','DATE'], axis=1, inplace = True)

week_train = week_train.groupby(['YEAR_WEEK','ID']).agg({ 
                                     'DELTA':sum,
                                     'PRECIPITATIONS':np.mean,
                                     'MIN_TEMP':min,
                                     'MEAN_TEMP':np.mean,
                                     'MAX_TEMP':max,
                                     'SUN':np.mean,
                                     'MEAN_CONSUMPTION':np.mean,
                                     'VARIANCE_CONSUMPTION':np.mean}).reset_index()

weeks_in_a_year = 50
week_train['sin_YEAR_WEEK'] = np.sin(2*np.pi*week_train['YEAR_WEEK']/weeks_in_a_year)
week_train['cos_YEAR_WEEK'] = np.cos(2*np.pi*week_train['YEAR_WEEK']/weeks_in_a_year) 
week_train.head()

,YEAR_WEEK,ID,DELTA,PRECIPITATIONS,MIN_TEMP,MEAN_TEMP,MAX_TEMP,SUN,MEAN_CONSUMPTION,VARIANCE_CONSUMPTION,sin_YEAR_WEEK,cos_YEAR_WEEK
0,0,0,1789.0,0.0,4.1,13.0,22.8,8.285714,282.893151,11453.639651,0.0,1.0
1,0,1,116.0,0.0,4.1,13.0,22.8,8.285714,11.690411,960.421214,0.0,1.0
2,0,2,0.0,0.0,4.1,13.0,22.8,8.285714,14.731507,363.554087,0.0,1.0
3,0,3,2698.0,0.0,4.1,13.0,22.8,8.285714,443.627397,25350.257948,0.0,1.0
4,0,4,2043.0,0.0,4.1,13.0,22.8,8.285714,265.456329,6416.593867,0.0,1.0


In [147]:
X_train = week_train.drop(['DELTA','YEAR_WEEK'], axis=1)
y_train = week_train['DELTA']

model1 = xgb.XGBRegressor(
    n_estimators=1000,
    reg_lambda=1,
    gamma=0,
    max_depth=8
)
model2 = GradientBoostingRegressor()

print('Fitting XGB...')
model1.fit(X_train, y_train)
print('Fitting GB...')
model2.fit(X_train, y_train)
print('End fitting.')

Fitting XGB...
Fitting GB...
End fitting.


In [148]:
path = '../data/df6.pkl'

df = pd.read_pickle(path)
start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
week_test = df[df['DATE'].isin(get_date_range(start_date, end_date))]
week_test['YEAR_WEEK'] = (week_test['YEAR_DAY']-1)//7
week_test = week_test[week_test['YEAR_WEEK']!=-1]
week_test.drop(['YEAR_DAY','IS_WEEKEND','WEEKDAY','sin_WEEKDAY','cos_WEEKDAY',
                 'sin_year_day','cos_year_day','IS_GOOD','DATE'], axis=1, inplace = True)

week_test = week_test.groupby(['YEAR_WEEK','ID']).agg({ 
                                     'DELTA':sum,
                                     'PRECIPITATIONS':np.mean,
                                     'MIN_TEMP':min,
                                     'MEAN_TEMP':np.mean,
                                     'MAX_TEMP':max,
                                     'SUN':np.mean,
                                     'MEAN_CONSUMPTION':np.mean,
                                     'VARIANCE_CONSUMPTION':np.mean}).reset_index()

weeks_in_a_year = 50
week_test['sin_YEAR_WEEK'] = np.sin(2*np.pi*week_test['YEAR_WEEK']/weeks_in_a_year)
week_test['cos_YEAR_WEEK'] = np.cos(2*np.pi*week_test['YEAR_WEEK']/weeks_in_a_year) 

X_test = week_test.drop(['DELTA','YEAR_WEEK'], axis=1)
y_test = week_test['DELTA']

In [149]:
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)

In [150]:
week_test['prediction1'] = y_pred1
week_test['prediction2'] = y_pred2
week_test['prediction'] = (week_test['prediction1'] + week_test['prediction2'])/2

In [151]:
final_df2['Semana_1'] = week_test[week_test['YEAR_WEEK']==50]['prediction']
final_df2['Semana_2'] = week_test[week_test['YEAR_WEEK']==51]['prediction']

test = pd.read_pickle('../data/test.pkl')
error = compute_error(final_df2, test)
print('Mean between XGBR and GBR:', round(error,2))

Mean between XGBR and GBR: 41706.25


In [144]:
week_test

,YEAR_WEEK,ID,DELTA,PRECIPITATIONS,MIN_TEMP,MEAN_TEMP,MAX_TEMP,SUN,MEAN_CONSUMPTION,VARIANCE_CONSUMPTION,sin_YEAR_WEEK,cos_YEAR_WEEK,prediction
0,50,0,2451.000000,23.216667,5.0,10.514286,18.8,1.185714,282.893151,1.145364e+04,-2.449294e-16,1.000000,1936.286987
1,50,1,241.670000,23.216667,5.0,10.514286,18.8,1.185714,11.690411,9.604212e+02,-2.449294e-16,1.000000,110.653191
2,50,2,234.000000,23.216667,5.0,10.514286,18.8,1.185714,14.731507,3.635541e+02,-2.449294e-16,1.000000,167.886551
3,50,3,2485.000000,23.216667,5.0,10.514286,18.8,1.185714,443.627397,2.535026e+04,-2.449294e-16,1.000000,2625.137207
4,50,4,2266.000000,23.216667,5.0,10.514286,18.8,1.185714,265.456329,6.416594e+03,-2.449294e-16,1.000000,2052.067139
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5489,51,2746,4080.533333,0.000000,7.6,15.728571,23.7,4.528571,582.933333,1.000229e+03,1.253332e-01,0.992115,3977.050781
5490,51,2747,2812.730452,0.000000,7.6,15.728571,23.7,4.528571,616.230144,1.477866e+06,1.253332e-01,0.992115,2486.467529
5491,51,2748,3050.817568,0.000000,7.6,15.728571,23.7,4.528571,589.401194,1.207967e+05,1.253332e-01,0.992115,2919.655029
5492,51,2749,817.515152,0.000000,7.6,15.728571,23.7,4.528571,116.787879,2.566854e-23,1.253332e-01,0.992115,806.589355


### Incluyendo lag_7

In [171]:
path = '../data/df6.pkl'

df = pd.read_pickle(path)
df['LAG_7'] = df['DELTA'].shift(7, fill_value=0)
df['LAG_14'] = df['DELTA'].shift(14, fill_value=0)

start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 17)
train = df[df['DATE'].isin(get_date_range(start_date, end_date))]
train = train[train['IS_GOOD']==1]
train.drop(['YEAR_DAY','WEEKDAY','IS_GOOD','DATE'], axis=1, inplace=True)
train['SUN'] = train['SUN'].fillna(train['SUN'].mean())
train['PRECIPITATIONS'] = train['PRECIPITATIONS'].fillna(train['PRECIPITATIONS'].mean())

start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 24)
test = df[df['DATE'].isin(get_date_range(start_date, end_date))]
test = test[test['IS_GOOD']==1]
test.drop(['YEAR_DAY','WEEKDAY','IS_GOOD'], axis=1, inplace=True)
test['SUN'] = test['SUN'].fillna(test['SUN'].mean())
test['PRECIPITATIONS'] = test['PRECIPITATIONS'].fillna(test['PRECIPITATIONS'].mean())

start_date = datetime.date(2020, 1 , 25)
end_date = datetime.date(2020, 1, 31)
test_2 = df[df['DATE'].isin(get_date_range(start_date, end_date))]
test_2 = test_2[test_2['IS_GOOD']==1]
test_2.drop(['YEAR_DAY','WEEKDAY','IS_GOOD'], axis=1, inplace=True)
test_2['SUN'] = test_2['SUN'].fillna(test_2['SUN'].mean())
test_2['PRECIPITATIONS'] = test_2['PRECIPITATIONS'].fillna(test['PRECIPITATIONS'].mean())

print('Train:', train.shape, 'Test:', test.shape, 'Test 2:', test_2.shape)

X_train = train.drop(['DELTA'], axis=1)
y_train = train['DELTA']

X_test = test.drop(['DELTA', 'DATE'], axis=1)
X_test_2 = test_2.drop(['DELTA', 'DATE'], axis=1)

Train: (931203, 16) Test: (18571, 17) Test 2: (18571, 17)


In [159]:
model1 = xgb.XGBRegressor(
    n_estimators=1000,
    reg_lambda=1,
    gamma=0,
    max_depth=8
)

model2 = GradientBoostingRegressor()

print('Fitting XGB...')
model1.fit(X_train, y_train)
print('Fitting GB...')
model2.fit(X_train, y_train)
print('End fitting.')

Fitting XGB...
Fitting GB...
End fitting.


In [196]:
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)

results_df = pd.DataFrame.from_dict({'ID':test['ID'].values, 
                                     'DATE':test['DATE'].values,
                                     'y_pred1':y_pred1,
                                     'y_pred2':y_pred2})
results_df = results_df.sort_values(['ID','DATE'])
results_df['FINAL'] = results_df[['y_pred1','y_pred2']].mean(axis=1)

X_test_2['LAG_7'] = results_df['FINAL'].values

y_pred1_2 = model1.predict(X_test_2)
y_pred2_2 = model2.predict(X_test_2)

results_df_2 = pd.DataFrame.from_dict({'ID':test_2['ID'].values, 
                                     'DATE':test_2['DATE'].values,
                                     'y_pred1':y_pred1,
                                     'y_pred2':y_pred2})
results_df_2 = results_df_2.sort_values(['ID','DATE'])
results_df_2['FINAL'] = results_df_2[['y_pred1','y_pred2']].mean(axis=1)

In [197]:
results_df = pd.concat([results_df, results_df_2])

In [200]:
start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
fechas_test = get_date_range(start_date, end_date)

ID = []
Dia_1 = []
Dia_2 = []
Dia_3 = []
Dia_4 = []
Dia_5 = []
Dia_6 = []
Dia_7 = []
for i, fecha in enumerate(fechas_test[0:7]):
    aux = results_df[results_df['DATE']==fecha]
    ID = list(aux['ID'].values)
    if i==0:
        Dia_1 += list(aux['FINAL'].values)
    if i==1:
        Dia_2 += list(aux['FINAL'].values)
    if i==2:
        Dia_3 += list(aux['FINAL'].values)
    if i==3:
        Dia_4 += list(aux['FINAL'].values)
    if i==4:
        Dia_5 += list(aux['FINAL'].values)
    if i==5:
        Dia_6 += list(aux['FINAL'].values)
    if i==6:
        Dia_7 += list(aux['FINAL'].values)
print(len(ID),len(Dia_1))
final_df = pd.DataFrame.from_dict({'ID':ID,
                                   'Dia_1':Dia_1,
                                  'Dia_2':Dia_2,
                                  'Dia_3':Dia_3,
                                  'Dia_4':Dia_4,
                                  'Dia_5':Dia_5,
                                  'Dia_6':Dia_6,
                                  'Dia_7':Dia_7,})

ID = []
Dia_8 = []
Dia_9 = []
Dia_10 = []
Dia_11 = []
Dia_12 = []
Dia_13 = []
Dia_14 = []
for i, fecha in enumerate(fechas_test[7:14]):
    aux = results_df[results_df['DATE']==fecha]
    ID = list(aux['ID'].values)
    if i==0:
        Dia_8 += list(aux['FINAL'].values)
    if i==1:
        Dia_9 += list(aux['FINAL'].values)
    if i==2:
        Dia_10 += list(aux['FINAL'].values)
    if i==3:
        Dia_11 += list(aux['FINAL'].values)
    if i==4:
        Dia_12 += list(aux['FINAL'].values)
    if i==5:
        Dia_13 += list(aux['FINAL'].values)
    if i==6:
        Dia_14 += list(aux['FINAL'].values)
print(len(ID),len(Dia_11))
final_df2 = pd.DataFrame.from_dict({'ID':ID,
                                   'Dia_8':Dia_8,
                                  'Dia_9':Dia_9,
                                  'Dia_10':Dia_10,
                                  'Dia_11':Dia_11,
                                  'Dia_12':Dia_12,
                                  'Dia_13':Dia_13,
                                  'Dia_14':Dia_14,})

final_df['Semana_1'] = final_df[['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']].sum(axis=1)
final_df['Semana_2'] = final_df2[['Dia_8','Dia_9','Dia_10','Dia_11','Dia_12','Dia_13','Dia_14']].sum(axis=1)

final_df2 = final_df.drop('ID', axis=1)

2653 2653
2653 2653


In [201]:
test = pd.read_pickle('../data/test.pkl')
error = compute_error(final_df2, test)
print('Mean between XGBR and GBR:', round(error,2))

Mean between XGBR and GBR: 15560.98


### Haciendo selección de variables 

In [208]:
for i in zip(X_train.columns, model1.feature_importances_):
    print(i)

('ID', 0.0037970373)
('MEAN_CONSUMPTION', 0.0024200587)
('VARIANCE_CONSUMPTION', 0.009339969)
('IS_WEEKEND', 0.0002044396)
('sin_WEEKDAY', 0.028839545)
('cos_WEEKDAY', 0.0020710642)
('sin_year_day', 0.005393776)
('cos_year_day', 0.00052915985)
('PRECIPITATIONS', 3.8060734e-05)
('MIN_TEMP', 0.8219458)
('MEAN_TEMP', 0.001233222)
('MAX_TEMP', 0.00012780189)
('SUN', 3.3295124e-05)
('LAG_7', 0.12373613)
('LAG_14', 0.00029065672)


In [216]:
best_vars = ['sin_WEEKDAY', 'MIN_TEMP', 'LAG_7']

In [217]:

path = '../data/df6.pkl'

df = pd.read_pickle(path)
df['LAG_7'] = df['DELTA'].shift(7, fill_value=0)
df['LAG_14'] = df['DELTA'].shift(14, fill_value=0)

start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 17)
train = df[df['DATE'].isin(get_date_range(start_date, end_date))]
train = train[train['IS_GOOD']==1]
train.drop(['YEAR_DAY','WEEKDAY','IS_GOOD','DATE'], axis=1, inplace=True)
train['SUN'] = train['SUN'].fillna(train['SUN'].mean())
train['PRECIPITATIONS'] = train['PRECIPITATIONS'].fillna(train['PRECIPITATIONS'].mean())

start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 24)
test = df[df['DATE'].isin(get_date_range(start_date, end_date))]
test = test[test['IS_GOOD']==1]
test.drop(['YEAR_DAY','WEEKDAY','IS_GOOD'], axis=1, inplace=True)
test['SUN'] = test['SUN'].fillna(test['SUN'].mean())
test['PRECIPITATIONS'] = test['PRECIPITATIONS'].fillna(test['PRECIPITATIONS'].mean())

start_date = datetime.date(2020, 1 , 25)
end_date = datetime.date(2020, 1, 31)
test_2 = df[df['DATE'].isin(get_date_range(start_date, end_date))]
test_2 = test_2[test_2['IS_GOOD']==1]
test_2.drop(['YEAR_DAY','WEEKDAY','IS_GOOD'], axis=1, inplace=True)
test_2['SUN'] = test_2['SUN'].fillna(test_2['SUN'].mean())
test_2['PRECIPITATIONS'] = test_2['PRECIPITATIONS'].fillna(test['PRECIPITATIONS'].mean())

print('Train:', train.shape, 'Test:', test.shape, 'Test 2:', test_2.shape)

X_train = train.drop(['DELTA'], axis=1)
y_train = train['DELTA']

X_test = test.drop(['DELTA', 'DATE'], axis=1)
X_test_2 = test_2.drop(['DELTA', 'DATE'], axis=1)

Train: (931203, 16) Test: (18571, 17) Test 2: (18571, 17)


In [218]:
model1 = xgb.XGBRegressor(
    n_estimators=1000,
    reg_lambda=1,
    gamma=0,
    max_depth=8
)

model2 = GradientBoostingRegressor()

print('Fitting XGB...')
model1.fit(X_train[best_vars], y_train)
print('Fitting GB...')
model2.fit(X_train[best_vars], y_train)
print('End fitting.')

Fitting XGB...
Fitting GB...
End fitting.


In [219]:
y_pred1 = model1.predict(X_test[best_vars])
y_pred2 = model2.predict(X_test[best_vars])

results_df = pd.DataFrame.from_dict({'ID':test['ID'].values, 
                                     'DATE':test['DATE'].values,
                                     'y_pred1':y_pred1,
                                     'y_pred2':y_pred2})
results_df = results_df.sort_values(['ID','DATE'])
results_df['FINAL'] = results_df[['y_pred1','y_pred2']].mean(axis=1)

X_test_2['LAG_7'] = results_df['FINAL'].values

y_pred1_2 = model1.predict(X_test_2[best_vars])
y_pred2_2 = model2.predict(X_test_2[best_vars])

results_df_2 = pd.DataFrame.from_dict({'ID':test_2['ID'].values, 
                                     'DATE':test_2['DATE'].values,
                                     'y_pred1':y_pred1,
                                     'y_pred2':y_pred2})
results_df_2 = results_df_2.sort_values(['ID','DATE'])
results_df_2['FINAL'] = results_df_2[['y_pred1','y_pred2']].mean(axis=1)

In [220]:
results_df = pd.concat([results_df, results_df_2])

In [221]:
#ESTO HAY QUE HACERLO MÁS LIMPIO 100%
start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
fechas_test = get_date_range(start_date, end_date)

final_df = pd.DataFrame(columns=['ID','Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7'], index=range(2653))

final_df['ID'] = results_df[results_df['DATE']==fechas_test[0]]['ID'].values
final_df['Dia_1'] = results_df[results_df['DATE']==fechas_test[0]]['FINAL'].values
final_df['Dia_2'] = results_df[results_df['DATE']==fechas_test[1]]['FINAL'].values
final_df['Dia_3'] = results_df[results_df['DATE']==fechas_test[2]]['FINAL'].values
final_df['Dia_4'] = results_df[results_df['DATE']==fechas_test[3]]['FINAL'].values
final_df['Dia_5'] = results_df[results_df['DATE']==fechas_test[4]]['FINAL'].values
final_df['Dia_6'] = results_df[results_df['DATE']==fechas_test[5]]['FINAL'].values
final_df['Dia_7'] = results_df[results_df['DATE']==fechas_test[6]]['FINAL'].values
final_df['Semana_1'] = final_df[['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']].sum(axis=1)

ID = []
Dia_8 = []
Dia_9 = []
Dia_10 = []
Dia_11 = []
Dia_12 = []
Dia_13 = []
Dia_14 = []
for i, fecha in enumerate(fechas_test[7:14]):
    aux = results_df[results_df['DATE']==fecha]
    ID = list(aux['ID'].values)
    if i==0:
        Dia_8 += list(aux['FINAL'].values)
    if i==1:
        Dia_9 += list(aux['FINAL'].values)
    if i==2:
        Dia_10 += list(aux['FINAL'].values)
    if i==3:
        Dia_11 += list(aux['FINAL'].values)
    if i==4:
        Dia_12 += list(aux['FINAL'].values)
    if i==5:
        Dia_13 += list(aux['FINAL'].values)
    if i==6:
        Dia_14 += list(aux['FINAL'].values)
print(len(ID),len(Dia_11))
final_df2 = pd.DataFrame.from_dict({'ID':ID,
                                   'Dia_8':Dia_8,
                                  'Dia_9':Dia_9,
                                  'Dia_10':Dia_10,
                                  'Dia_11':Dia_11,
                                  'Dia_12':Dia_12,
                                  'Dia_13':Dia_13,
                                  'Dia_14':Dia_14,})

final_df['Semana_1'] = final_df[['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']].sum(axis=1)
final_df['Semana_2'] = final_df2[['Dia_8','Dia_9','Dia_10','Dia_11','Dia_12','Dia_13','Dia_14']].sum(axis=1)

final_df2 = final_df.drop('ID', axis=1)

2653 2653
2653 2653


In [222]:
test = pd.read_pickle('../data/test.pkl')
error = compute_error(final_df2, test)
print('Mean between XGBR and GBR:', round(error,2))

Mean between XGBR and GBR: 3032.2


In [224]:
results_df.head()

,ID,DATE,y_pred1,y_pred2,FINAL
0,0,2020-01-18,316.047729,194.544385,255.296057
1,0,2020-01-19,277.047455,247.796851,262.422153
2,0,2020-01-20,148.628922,171.009133,159.819027
3,0,2020-01-21,403.525452,454.017490,428.771471
4,0,2020-01-22,219.860413,228.564084,224.212248


In [225]:
start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
fechas_test = get_date_range(start_date, end_date)

results_df[results_df['DATE']==fechas_test[0]]

,ID,DATE,y_pred1,y_pred2,FINAL
0,0,2020-01-18,316.047729,194.544385,255.296057
7,1,2020-01-18,143.123077,144.629730,143.876404
14,2,2020-01-18,186.664093,144.629730,165.646912
21,3,2020-01-18,493.544800,634.361280,563.953040
28,4,2020-01-18,373.401031,292.501093,332.951062
...,...,...,...,...,...
18536,2705,2020-01-18,141.381378,144.629730,143.005554
18543,2707,2020-01-18,141.381378,144.629730,143.005554
18550,2709,2020-01-18,344.005127,247.796851,295.900989
18557,2711,2020-01-18,130.533096,144.629730,137.581413


In [231]:
final_df = pd.DataFrame(columns=['ID','Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7'], index=range(2653))

final_df['ID'] = results_df[results_df['DATE']==fechas_test[0]]['ID'].values
final_df['Dia_1'] = results_df[results_df['DATE']==fechas_test[0]]['FINAL'].values
final_df['Dia_2'] = results_df[results_df['DATE']==fechas_test[1]]['FINAL'].values
final_df['Dia_3'] = results_df[results_df['DATE']==fechas_test[2]]['FINAL'].values
final_df['Dia_4'] = results_df[results_df['DATE']==fechas_test[3]]['FINAL'].values
final_df['Dia_5'] = results_df[results_df['DATE']==fechas_test[4]]['FINAL'].values
final_df['Dia_6'] = results_df[results_df['DATE']==fechas_test[5]]['FINAL'].values
final_df['Dia_7'] = results_df[results_df['DATE']==fechas_test[6]]['FINAL'].values
final_df['Semana_1'] = final_df[['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']].sum(axis=1)